# Constraint Extractor + Notion Constraint Memory Playground

This notebook is a hands-on playground for:

- Verifying which Notion page the timeboxing DBs are installed under
- Seeding default constraint types
- Running inserts (upserts) + retrievals (filter/search)
- Exercising the *LLM-backed* constraint extractor end-to-end

Everything goes through the `constraint-memory` MCP server (stdio): `scripts/constraint_mcp_server.py`.

## Prereqs

- `NOTION_TOKEN`
- `NOTION_TIMEBOXING_PARENT_PAGE_ID`
- LLM creds (`OPENAI_API_KEY` or `OPENROUTER_API_KEY`, depending on `LLM_PROVIDER`) if you run the extractor section

Tip: if you're using the repo's `.env`, make sure your Jupyter kernel is running in the repo's virtualenv.


In [ ]:
import json
import os
from datetime import date

import nest_asyncio
from dotenv import find_dotenv, load_dotenv

nest_asyncio.apply()
load_dotenv(find_dotenv(), override=False)

# Safety switches
CONFIRM_WRITES = False  # set True to allow upserts/logging to Notion
CONFIRM_EXTRACTOR = False  # set True to allow LLM extractor call (also writes to Notion)

parent_page_id = (os.getenv("NOTION_TIMEBOXING_PARENT_PAGE_ID") or "").strip()
print("NOTION_TIMEBOXING_PARENT_PAGE_ID:", parent_page_id)
print("NOTION_TOKEN set:", bool((os.getenv("NOTION_TOKEN") or "").strip()))


In [ ]:
from fateforger.tools.constraint_mcp import get_constraint_mcp_tools

tools = await get_constraint_mcp_tools(timeout=15.0)
print(f"Loaded {len(tools)} MCP tools")
for t in tools:
    print("-", t.name)


In [ ]:
def _tool_by_name(name: str):
    for t in tools:
        if t.name == name:
            return t
    raise KeyError(f"Tool not found: {name}")


async def call_tool(name: str, **kwargs):
    tool = _tool_by_name(name)
    return await tool.run_json(kwargs)


def jdump(obj) -> str:
    return json.dumps(obj, indent=2, ensure_ascii=False, sort_keys=True)


In [ ]:
# Shows where the DBs are installed + each DB id/url
info = await call_tool("constraint.get_store_info")
print(jdump(info))


In [ ]:
# One-time seeding of default types (idempotent)
if not CONFIRM_WRITES:
    raise RuntimeError("Set CONFIRM_WRITES=True to seed types (writes to Notion).")

result = await call_tool("constraint.seed_types")
print(result)


In [ ]:
# Upsert a couple demo constraints (idempotent if you keep the same UIDs)
if not CONFIRM_WRITES:
    raise RuntimeError("Set CONFIRM_WRITES=True to allow upserts (writes to Notion).")

today = date.today().isoformat()

demo_constraints = [
    {
        "constraint_record": {
            "name": "Prefer Deep Work in the Morning",
            "description": "Prefer scheduling deep work between 09:00 and 12:00 on weekdays.",
            "necessity": "should",
            "status": "proposed",
            "source": "user",
            "confidence": 0.9,
            "scope": "profile",
            "applicability": {
                "days_of_week": ["MO", "TU", "WE", "TH", "FR"],
                "timezone": "Europe/Amsterdam",
            },
            "lifecycle": {"uid": "tb:demo:prefer_window:deep_work_morning"},
            "payload": {
                "rule_kind": "prefer_window",
                "scalar_params": {"contiguity": "prefer"},
                "windows": [
                    {"kind": "prefer", "start_time_local": "09:00", "end_time_local": "12:00"}
                ],
            },
            "applies_stages": ["Skeleton", "Refine", "ReviewCommit"],
            "applies_event_types": ["DW"],
            "topics": ["deep work", "focus"],
        }
    },
    {
        "constraint_record": {
            "name": "Avoid Meetings After 16:00",
            "description": "Avoid scheduling meetings after 16:00 unless unavoidable.",
            "necessity": "should",
            "status": "proposed",
            "source": "user",
            "confidence": 0.75,
            "scope": "profile",
            "applicability": {
                "days_of_week": ["MO", "TU", "WE", "TH", "FR"],
                "timezone": "Europe/Amsterdam",
            },
            "lifecycle": {"uid": "tb:demo:avoid_window:meetings_late"},
            "payload": {
                "rule_kind": "avoid_window",
                "windows": [
                    {"kind": "avoid", "start_time_local": "16:00", "end_time_local": "18:30"}
                ],
            },
            "applies_stages": ["Skeleton", "Refine", "ReviewCommit"],
            "applies_event_types": ["M"],
            "topics": ["meetings"],
        }
    },
]

upsert_results = []
for record in demo_constraints:
    event = {
        "user_utterance": f"[notebook demo] upsert @ {today}",
        "triggering_suggestion": None,
        "stage": "CollectConstraints",
        "event_types": (record["constraint_record"].get("applies_event_types") or []),
        "decision_scope": "other",
        "action": "upsert",
        "overrode_planner": False,
        "extraction_confidence": record["constraint_record"].get("confidence"),
        "extracted_type_id": record["constraint_record"].get("payload", {}).get("rule_kind"),
    }
    res = await call_tool("constraint.upsert_constraint", record=record, event=event)
    upsert_results.append(res)

print(jdump(upsert_results))


In [ ]:
# Query + filter/search (this is the main "search/filter tool" demo)

filters = {
    "as_of": date.today().isoformat(),
    "require_active": True,
    "scopes_any": ["profile"],
}

all_profile = await call_tool(
    "constraint.query_constraints",
    filters=filters,
    sort=[["Confidence", "desc"], ["Name", "asc"]],
    limit=25,
)
print("--- profile constraints (sorted) ---")
print(jdump(all_profile))

deep_work = await call_tool(
    "constraint.query_constraints",
    filters={"as_of": date.today().isoformat(), "text_query": "Deep Work"},
    limit=10,
)
print("\n--- text_query=Deep Work ---")
print(jdump(deep_work))

tagged = await call_tool(
    "constraint.query_constraints",
    filters={"as_of": date.today().isoformat()},
    tags=["deep work"],
    limit=10,
)
print("\n--- tags=['deep work'] ---")
print(jdump(tagged))

prefer_window_only = await call_tool(
    "constraint.query_constraints",
    filters={"as_of": date.today().isoformat()},
    type_ids=["prefer_window"],
    limit=10,
)
print("\n--- type_ids=['prefer_window'] ---")
print(jdump(prefer_window_only))


In [ ]:
# Fetch a single constraint by UID
uid = "tb:demo:prefer_window:deep_work_morning"
one = await call_tool("constraint.get_constraint", uid=uid)
print(jdump(one))


In [ ]:
# LLM-backed extractor (calls MCP tools to upsert into Notion)
# This will make an LLM API call and write to Notion.

if not CONFIRM_EXTRACTOR:
    raise RuntimeError("Set CONFIRM_EXTRACTOR=True to run the extractor (LLM call + Notion writes).")

from fateforger.agents.timeboxing.notion_constraint_extractor import NotionConstraintExtractor
from fateforger.llm import build_autogen_chat_client

model_client = build_autogen_chat_client("timeboxing_agent", parallel_tool_calls=False)
extractor = NotionConstraintExtractor(model_client=model_client, tools=tools)

output = await extractor.extract_and_upsert_constraint(
    planned_date=date.today().isoformat(),
    timezone="Europe/Amsterdam",
    stage_id="CollectConstraints",
    user_utterance="In general, I prefer deep work between 09:00 and 12:00 on weekdays.",
    impacted_event_types=["DW"],
    suggested_tags=["deep work"],
    decision_scope="place_dw_blocks",
)

print("--- extractor output ---")
print(jdump(output))

uid = None
try:
    uid = (
        (output or {})
        .get("constraint_record", {})
        .get("lifecycle", {})
        .get("uid")
    )
except Exception:
    uid = None

if uid:
    print("\n--- fetched by uid ---")
    print(jdump(await call_tool("constraint.get_constraint", uid=uid)))
else:
    name = ((output or {}).get("constraint_record", {}) or {}).get("name")
    if name:
        print("\n--- uid not present; searching by name ---")
        matches = await call_tool(
            "constraint.query_constraints",
            filters={"as_of": date.today().isoformat(), "text_query": name},
            limit=10,
        )
        print(jdump(matches))
